In [7]:
import numpy as np

In [50]:
INTENT_INDICES = range(0,19)
TAGS_INDICES   = range(19,30)

# Apply two logics for Multiclass Columns and Multilabel Columns
def get_preds_from_logits(logits):
    ret = np.zeros(logits.shape)

    # The first 19 columns are for customer intents. They should be handled with a multiclass approach
    # i.e. we fill 1 to the class with highest probability, and 0 into the other columns
    best_class = np.argmax(logits[:, INTENT_INDICES], axis=-1)
    ret[list(range(len(ret))), best_class] = 1

    # The other columns are for register tags. They should be handled with multilabel approach.
    # i.e. we fill 1 to every class whose score is higher than some threshold
    # In this example, we choose that threshold = 0
    ret[:, TAGS_INDICES] = (logits[:, TAGS_INDICES] >= 0).astype(int)

    return ret

In [2]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

/home/levis/Others/Projects/customer-support-intent-linguistic-prediction/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
import os

# Load the custom pretrained model

#navigate to parent directory
parent_dir = os.path.dirname(os.getcwd())

#navigate to model directory
model_path = os.path.join(parent_dir, 'models')

model = DistilBertForSequenceClassification.from_pretrained(model_path)

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [17]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [18]:
model = model.to(device)

In [33]:
input_texts =  "i have bought the same item twice cancel order 00123842"

In [34]:
# Encode the text
encoded_input = tokenizer(input_texts,
                          truncation=True,
                          padding="max_length",
                          max_length=128,
                          return_tensors='pt').to('cuda')

In [35]:
# Call the model to predict under the format of logits of 30 classes
logits = model(**encoded_input).logits.cpu().detach().numpy()

In [36]:
# Decode the result
preds = get_preds_from_logits(logits)

In [37]:
preds

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0.]])

## TRANSLATING OUTPUTS FROM OUTPUT LABELS

In [9]:
# LABEL DICTIONARY

intents_map = {
    0: 'cancel_order',
    1: 'change_order',
    2: 'change_shipping_address',
    3: 'check_payment_methods',
    4: 'check_refund_policy',
    5: 'contact_customer_service',
    6: 'contact_human_agent',
    7: 'create_account',
    8: 'delete_account',
    9: 'edit_account',
   10: 'get_refund',
   11: 'payment_issue',
   12: 'place_order',
   13: 'recover_password',
   14: 'registration_problems',
   15: 'set_up_shipping_address',
   16: 'switch_account',
   17: 'track_order',
   18: 'track_refund'
}

tags_labels = np.array(['Q', 'P', 'W', 'K', 'B', 'C', 'I', 'M', 'L', 'E', 'Z'])

In [41]:
intents_array = preds[0][:19]
tags_array = preds[0][19:]

In [40]:
intents_array

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [42]:
tags_array

array([0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0.])

### Intent Prediction

In [46]:
intent = intents_map[intents_array.argmax()]

In [47]:
intent

'cancel_order'

### Language Prediction

In [48]:
output_tag = ""

for i in np.where(tags_array == 1)[0]:
  output_tag += tags_labels[i]

In [49]:
output_tag

'BCL'